# 과제 - 3주차

In [ ]:
from pyspark.sql import SparkSession

# SparkSession
spark = (
    SparkSession.builder
        .appName("rdd-dataframe")
        .master("local")
        .getOrCreate()
)

# SparkContext를 SparkSession에서 빼두기
sc = spark.sparkContext

# 하단 메시지는 Jupyter 공식이미지에서 나오는 메시지로, 무시하기

## 1. spark-submit.sh 스크립트 참고해서 option 바꾸어보기

## 2. 원하는 최종 스키마를 만들기 위한 정제 코드 filter.py 작성하기

In [ ]:
# json파일로 저장해 둔 스키마 불러오기

import json
from pyspark.sql.types import StructType

github_schema = json.loads(github.schema.json())

schema_to_read = StructType.fromJson(github_schema)
df = spark.read.schema(schema_to_read).json("../data/*.json.gz")

In [ ]:
# 스키마 확인하기

def find_all_names(data, current_path=[]):
    paths = []
    if isinstance(data, dict):
        if 'name' in data:
            # Collect the path to the current 'name'
            paths.append('.'.join(current_path + [data['name']]))
        # Recursively search within this dictionary
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                paths.extend(find_all_names(value, current_path + [data.get('name', key)]))
    elif isinstance(data, list):
        for item in data:
            paths.extend(find_all_names(item, current_path))
    return paths

for each_line in find_all_names(github_schema):
    print(each_line.replace('fields.',''))

In [8]:
# 데이터 확인

import pyspark.sql.functions as F

columns = ['actor.avatar_url','actor.id','created_at']
select_exprs = [F.col(col_path).alias(col_path) for col_path in columns]

df.select(*select_exprs).show(3)

+--------------------+--------+--------------------+
|    actor.avatar_url|actor.id|          created_at|
+--------------------+--------+--------------------+
|https://avatars.g...|83618436|2024-05-19T14:00:00Z|
|https://avatars.g...|  132692|2024-05-19T14:00:00Z|
|https://avatars.g...|78722358|2024-05-19T14:00:00Z|
+--------------------+--------+--------------------+
only showing top 3 rows



## 3. 작성한 정제 코드 spark-submit 해보기
* 제출할 내용: spark-submit.sh, filter.py (파일명은 바꾸셔도 무방합니다)